In [1]:
import pickle

train_name = 'train_untrained_bert_tabular.p'
test_name = 'test_untrained_bert_tabular.p'

with open(train_name, 'rb') as fp:
    train_data = pickle.load(fp)

with open(test_name, 'rb') as fp:
    test_data = pickle.load(fp)
    
with open('test_comps_tabular.p', 'rb') as fp:
    test_comps = pickle.load(fp)
    
with open('enc_states_sync_all.p', 'rb') as fp:
    all_comps = pickle.load(fp)

In [2]:
import pandas as pd
import numpy as np

graph = pd.read_csv('vertices.csv')

meta = pd.read_csv('kernels_meta.csv')

In [3]:
graph

,graph_vertex_id,graph_vertex_class,graph_vertex_subclass
0,1,Hyperparam_Tuning,find_best_score
1,2,Hyperparam_Tuning,find_best_params
2,3,Hyperparam_Tuning,find_best_model_class
3,4,Model_Train,choose_model_class
4,5,Hyperparam_Tuning,define_search_space
...,...,...,...
70,88,Debug,list_files
71,91,Data_Extraction,prepare_data_loader
72,92,Visualization,plot_metrics
73,93,Visualization,images


In [4]:
# for sample in test_data:
#     print([graph[graph.graph_vertex_id == x]['graph_vertex_subclass'].values[0] for x in sample['target'].numpy() if x < 95])

In [5]:
train_data[0].keys()

dict_keys(['kernel_id', 'encoded_sequence', 'encoded_tokens', 'target', 'classes', 'comp_name', 'weight'])

In [6]:
train_data[0]["encoded_sequence"].shape

torch.Size([788])

In [7]:
np.max([len(note["target"]) for note in train_data])

125

In [8]:
np.max([len(note["target"]) for note in test_data])

139

In [9]:
val = set()

for note in train_data:
    val = val | set(note["target"].numpy())
    
for note in test_data:
    val = val | set(note["target"].numpy())

val = list(val)
print(val)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 25, 26, 28, 34, 35, 38, 40, 41, 42, 43, 44, 45, 47, 48, 52, 53, 54, 57, 58, 61, 62, 66, 68, 69, 75, 76]


In [10]:
len(val)

50

In [11]:
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import mean_absolute_error as MAE

def test(X_train, X_test, y_train, y_test):
    pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('model', Ridge())
    ])

    pipe.fit(X_train, y_train)
    print("R^2:", pipe.score(X_test, y_test))
    print("MSE:", MSE(pipe.predict(X_test), y_test))
    print("MAE:", MAE(pipe.predict(X_test), y_test))

In [12]:
X_train, X_test, y_train, y_test = [], [], [], []

for note in train_data:
    score = meta[meta.kernel_id == note["kernel_id"]].kaggle_score.iloc[0]
    X_train.append(note["encoded_sequence"].numpy())
    y_train.append(score)
        
for note in test_data:
    score = meta[meta.kernel_id == note["kernel_id"]].kaggle_score.iloc[0]
    X_test.append(note["encoded_sequence"].numpy())
    y_test.append(score)
    
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

test(X_train, X_test, y_train, y_test)

R^2: 0.9702744177313025
MSE: 5278753573.952385
MAE: 9446.60715717088


In [13]:
X_train, X_test, y_train, y_test = [], [], [], []

for note in train_data:
    score = meta[meta.kernel_id == note["kernel_id"]].kaggle_score.iloc[0]
    if score >= 1:
        continue
    X_train.append(note["encoded_sequence"].numpy())
    y_train.append(score)
        
for note in test_data:
    score = meta[meta.kernel_id == note["kernel_id"]].kaggle_score.iloc[0]
    if score >= 1:
        continue
    X_test.append(note["encoded_sequence"].numpy())
    y_test.append(score)
    
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

test(X_train, X_test, y_train, y_test)

R^2: 0.9406659012905054
MSE: 0.009405832059876109
MAE: 0.02587763001440488


In [14]:
X_train, X_test, y_train, y_test = [], [], [], []

for note in train_data:
    score = meta[meta.kernel_id == note["kernel_id"]].kaggle_score.iloc[0]
    if score >= 1:
        continue
    X_train.append(np.append(note["encoded_sequence"].numpy(), len(note["target"])))
    y_train.append(score)
        
for note in test_data:
    score = meta[meta.kernel_id == note["kernel_id"]].kaggle_score.iloc[0]
    if score >= 1:
        continue
    X_test.append(np.append(note["encoded_sequence"].numpy(), len(note["target"])))
    y_test.append(score)
    
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

test(X_train, X_test, y_train, y_test)

R^2: 0.9411299821051287
MSE: 0.009332264477330893
MAE: 0.02586866078512514


In [15]:
max_len = 50

X_train, X_test, y_train, y_test = [], [], [], []

for note in train_data:
    score = meta[meta.kernel_id == note["kernel_id"]].kaggle_score.iloc[0]
    
    target = note["target"]
    target_len = len(target)
    target = target[:max_len]
    target = np.pad(target, (0, max_len - len(target)),  "constant", constant_values = -1)
    
    X_train.append(np.append(
        note["encoded_sequence"], np.append(target, target_len)
    ))
    
    y_train.append(score)
    
    bad_target_len = np.random.randint(max_len)
    bad_target = np.random.choice(val, bad_target_len, True)
    bad_target = np.pad(bad_target, (0, max_len - len(bad_target)),  "constant", constant_values = -1)
    
    X_train.append(np.append(
        note["encoded_sequence"], np.append(bad_target, bad_target_len)
    ))
    
    if score < 1:
        y_train.append(np.random.uniform(0.0, 0.01))
    else:
        y_train.append(np.random.uniform(0.0, 1000))
        
for note in test_data:
    score = meta[meta.kernel_id == note["kernel_id"]].kaggle_score.iloc[0]
    
    target = note["target"]
    target_len = len(target)
    target = target[:max_len]
    target = np.pad(target, (0, max_len - len(target)),  "constant", constant_values = -1)
    
    X_test.append(np.append(
        note["encoded_sequence"], np.append(target, target_len)
    ))
    
    y_test.append(score)
    
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

test(X_train, X_test, y_train, y_test)

R^2: 0.7587831692422214
MSE: 42835971923.112366
MAE: 41654.77639876845


In [16]:
scores = dict()

for note in train_data:
    func = note["encoded_sequence"][-20:].argmax().item()
    score = meta[meta.kernel_id == note["kernel_id"]].kaggle_score.iloc[0]
    
    if scores.get(func) is None:
        scores[func] = []
    
    scores[func].append(score)

In [17]:
len(scores)

17

In [18]:
set(scores.keys())

{0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 13, 14, 15, 16, 17, 18, 19}

In [19]:
for func, values in scores.items():
    print(func)
    ser = pd.Series(values)
    print(ser.describe())
    print('=' * 50)

2
count    618.000000
mean       0.819965
std        0.168674
min        0.056730
25%        0.782000
50%        0.843500
75%        0.937975
max        1.000000
dtype: float64
16
count    563.000000
mean       0.461181
std        1.307298
min        0.000000
25%        0.121465
50%        0.130780
75%        0.475705
max       20.687910
dtype: float64
17
count    194.000000
mean       0.166770
std        0.255111
min        0.000000
25%        0.035160
50%        0.046000
75%        0.067290
max        0.751970
dtype: float64
6
count    224.000000
mean       0.305926
std        0.699558
min        0.017990
25%        0.018427
50%        0.018680
75%        0.019000
max        2.491360
dtype: float64
9
count    2.200000e+02
mean     2.913397e+05
std      1.214972e+06
min     -2.219330e+00
25%      3.084000e-02
50%      5.665000e-02
75%      1.507000e+00
max      5.715154e+06
dtype: float64
1
count    660.000000
mean       0.830646
std        0.612427
min        0.235290
25%        0.78

In [24]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(all_comps, test_size = 0.3, random_state = 14)

In [26]:
max_len = 50

X_train, X_test, y_train, y_test = [], [], [], []

for note in train_data:
    score = meta[meta.kernel_id == note["kernel_id"]].kaggle_score.iloc[0]
    
    if score >= 1:
        continue
    
    target = note["target"]
    target_len = len(target)
    target = target[:max_len]
    target = np.pad(target, (0, max_len - len(target)),  "constant", constant_values = -1)
    
    X_train.append(np.append(
        note["encoded_sequence"], np.append(target, target_len)
    ))
    
    y_train.append(score)
        
for note in test_data:
    score = meta[meta.kernel_id == note["kernel_id"]].kaggle_score.iloc[0]
    
    if score >= 1:
        continue
    
    target = note["target"]
    target_len = len(target)
    target = target[:max_len]
    target = np.pad(target, (0, max_len - len(target)),  "constant", constant_values = -1)
    
    X_test.append(np.append(
        note["encoded_sequence"], np.append(target, target_len)
    ))
    
    y_test.append(score)
    
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

test(X_train, X_test, y_train, y_test)

R^2: 0.933486117216911
MSE: 0.060104456466649754
MAE: 0.11319744169031715
